In [9]:
# from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from glob import glob

In [10]:
# 종목 뉴스리스트 페이지별로 전체 html코드를 긁어온 후, 리스트 형식으로 반환
def getData(codeList, newsPage):
    pageList = []
    for code in codeList:
        for page in range(newsPage, newsPage+1):
            newsListPageUrl = 'https://finance.naver.com/item/news_news.naver?code='+str(code)+'&page='+str(page)+'&sm=title_entity_id.basic&clusterId='
            data = requests.get(newsListPageUrl)
            soup = BeautifulSoup(data.content.decode('euc-kr', 'replace'), 'html.parser')
            dataFilter = soup.find('tbody')
            pageList.append(dataFilter)
    return pageList

In [11]:
# 뉴스페이지별로 전체 html코드를 가공하여 각 뉴스기사들의 url주소를 만들어서, 리스트 형식으로 반환
def change(bs4Data, num):
    makeUrlList = []
    newsUrlList = []
    for bs4 in bs4Data:
        change = bs4.select('tbody > tr > td > a')
        for i in range(len(change)):
            if num <= 9:
                newsUrlList.append('https://finance.naver.com'+str(change[i])[21:65]+str(change[i])[69:83]+str(change[i])[87:99]+str(change[i])[103:110]+str(change[i])[114:138])
            elif num <= 99:
                newsUrlList.append('https://finance.naver.com'+str(change[i])[21:65]+str(change[i])[69:83]+str(change[i])[87:99]+str(change[i])[103:111]+str(change[i])[115:139])
            elif num <= 999:
                newsUrlList.append('https://finance.naver.com'+str(change[i])[21:65]+str(change[i])[69:83]+str(change[i])[87:99]+str(change[i])[103:112]+str(change[i])[116:140])
    return newsUrlList

In [12]:
def getNewsData(urlLists):
    newsDateList = []
    newsTitleList = []
    newsTextList = []
    for i in range(len(urlLists)):
        html = requests.get(urlLists[i])
        soup = BeautifulSoup(html.content.decode('euc-kr', 'replace'), 'html.parser')
        newsDateList.append(soup.find('span',{'class':'tah p11'}).get_text())
        newsTitleList.append(soup.find('strong', {'class':'c p15'}).get_text())
        newsText = soup.find('div', {'class':'scr01'})
        if newsText.find('div', {'class':'link_news'}) != None:
            newsText.find('div', {'class':'link_news'}).decompose()
        newsTextList.append(newsText.get_text()[1:-1])
    newsDf = pd.DataFrame({
        '날짜':newsDateList
        , '제목':newsTitleList
        , '내용':newsTextList
    })
    return newsDf

In [13]:
def filescan():
    fileDataList = []
    fileList = glob('../뉴스데이터 수집/*.csv')
    for i in range(len(fileList)):
        fileDataList.append(pd.read_csv('../뉴스데이터 수집//'+fileList[i][11:], encoding='utf-8'))
    hapData = pd.concat(fileDataList)
    hapData.drop_duplicates(['제목'], inplace=True)
    hapData.sort_values(['날짜'], inplace=True)
    hapData.reset_index(inplace=True)
    hapData.drop('index', axis=1, inplace=True)
    hapData.to_csv('../뉴스데이터 수집/AI 뉴스 데이터.csv', encoding='utf-8', index=False)

In [14]:
def openFile():
    filtering = pd.read_csv('../뉴스데이터 수집/AI 뉴스 데이터.csv', encoding='utf-8')
    for i in range(0, len(filtering)):
        filtering['날짜'][i] = filtering['날짜'][i].strip()
        filtering['내용'][i] = str(filtering['내용'][i]).replace('\t','')
    filtering.to_csv('../뉴스데이터 수집/AI 뉴스 데이터.csv', encoding='utf-8', index=False)
    return pd.read_csv('../뉴스데이터 수집//AI 뉴스 데이터.csv', encoding='utf-8')

In [15]:
stockCodeList = ['108860','079970', '300080', '338220', '347860'] # 원하는 종목
stockNewsPage = 999        # 원하는 뉴스페이지 숫자 1 ~ 999페이지까지만 긁어오기 가능

for numberCount in range(1, stockNewsPage):
    bs4Data = getData(stockCodeList, numberCount)
    urlList = change(bs4Data, numberCount)
    getNewsDf = getNewsData(urlList)
    getNewsDf.to_csv('../뉴스데이터 수집/AI 뉴스 데이터'+str(numberCount)+'.csv', encoding='utf-8', index=False) # 원하는 종목 코드에 맞는 종목 이름으로 꼭 바꿔서 사용하세요.

# 진행도는 여기서 확인할 수 없으니, ./newsdata/안에 있는 파일 생성되는 것을 확인하세요. -> 네이버 기사 페이지 숫자가 파일명에 붙어서 만들어집니다.
# 예) 하이닉스 뉴스 데이터77.csv -> 하이닉스 뉴스 77페이지 뉴스 리스트 다 긁어옴
# 네이버 봇이 자동으로 막는 문제인지 알 수 없으나, 30~50개 정도 파일을 만들어낼때마다 'Attribute error'가 뜨는데
# 하이닉스 뉴스 데이터93.csv 까지 파일이 만들어졌다가 에러가 떳다면
# for numberCount in range(93, stockNewsPage) <- 이 코드에서 네이버 뉴스 페이지 수 93을 
# 입력한채로 다시 실행하면 다시 만들기가 시작됩니다.
# 에러가 안뜰 수도 있음.
# 테스트 결과 하이닉스 총 344페이지 뉴스 페이지 전부 가져와지는 것을 확인함.

filescan()
openFile()

OSError: Could not find a suitable TLS CA certificate bundle, invalid path: c:\ProgramData\anaconda3\envs\stock_info\lib\site-packages\certifi\cacert.pem